In [86]:
import pandas as pd

In [106]:
municipality = pd.read_csv("clean_data/turkey_municipality.csv")
parlimentary = pd.read_csv("clean_data/turkish_parlimentary.csv")
parlimentary_d = pd.read_csv("clean_data/milletvekili_detailed.csv")
party_alignment = pd.read_excel("clean_data/party_alignment.xlsx")

In [107]:
municipality.drop(columns = ['prev_year',
       'winner', "prev_winner", "swing"], inplace = True)

In [108]:
municipality.city = municipality.city.str.replace("mersin", "icel")

In [109]:
municipality = municipality[(municipality.city != "turkiye") & (municipality.city != "altkademebelediyesi")]
municipality["percent_turnout"] = municipality["turnout"] / municipality["registered_voters"]

In [110]:
def determine_winner(row):
    row = row.dropna(axis=0, how='any')
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [111]:
years = list(municipality.year.unique())

In [112]:
years = sorted(years, reverse = True)

In [113]:
municipality["prev_year"] = municipality.year.apply(lambda row: None if row == 1989 else years[years.index(row) + 1])

In [114]:
municipality["winner"] = municipality.apply(lambda row: determine_winner(row), axis = 1)
municipality["winner"] = municipality["winner"].str.split("_", expand = True)[0]

In [115]:
municipality_results = municipality[["city", "year", "winner"]].drop_duplicates()
municipality_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
municipality = municipality.merge(municipality_results, on = ["city", "prev_year"], how = "left")

In [116]:
municipality["swing"] = ((municipality["winner"] != municipality["prev_winner"]) & (municipality["prev_winner"].notna())).astype(int)

In [117]:
municipality.winner = municipality.winner.str.lower()
municipality.prev_winner = municipality.prev_winner.str.lower()
municipality.winner = municipality.winner.str.replace("ak parti", "akp")
municipality.prev_winner = municipality.prev_winner.str.replace("ak parti", "akp")

In [118]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
party_alignment.winner = party_alignment.winner.str.replace("ak parti", "akp")

In [119]:
municipality = municipality.merge(party_alignment, on = "winner", how = "left")

In [120]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]

In [121]:
municipality = municipality.merge(party_alignment, on = "prev_winner", how = "left")

In [123]:
municipality.to_csv("clean_data/turkey_municipality.csv", index = False)